In [1]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_BESTCASE_RESIDUAL, run_GaBP_HARDWARE_BESTCASE, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS_STOCHASTIC
# , run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_2_STREAMS
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult
 
import warnings
import matplotlib
import numpy as np
import math
import random
import os 

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

#### INTEL

In [2]:
# file_path = 'GBP/Raw_Datasets/data/input_M3500_g2o.g2o'
# data_gen.generate_SLAM_dataset(file_path=file_path)

# dataset = 'input_M3500_g2o'
# filepath_n = 'GBP/Raw_Datasets/gbp_data'

# factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
# marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')

In [3]:
# num_iter = 100
# it = 0
# max_iter = 10000
# sync_convergence_threshold = 1*10**-5
# while(it < num_iter):
#     print(f"------------ ITERATION {it} ------------")
#     data_gen.generate_SLAM_dataset(file_path=file_path)
#     A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)
#     P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=max_iter, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
#     print(f"number_iterations = {iteration}")
#     if iteration < max_iter:
#         print("------------ DONE ------------")
#         break
#     it += 1

## MITb

In [4]:
# file_path = 'GBP/Raw_Datasets/data/input_MITb_g2o.g2o'
# data_gen.generate_SLAM_dataset(file_path=file_path)

dataset = 'input_MITb_g2o'
filepath_n = 'GBP/Raw_Datasets/gbp_data'

factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')

In [5]:
ASYNC_ITER = 10
PEs = 128
NODE_UPDT_PE = 4

### MITb - Sync

In [6]:
A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)

# A, b = data_gen.get_1D_line_matrix(int(1000), scaling=True, normalized=False)

max_iter = 10000
sync_convergence_threshold = 1*10**-5
P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=max_iter, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
final_mean = list(mu_i)
print(f"number_iterations = {iteration}")

print(A.shape[0])

iteration: 1
0.030685077361966316
-----
iteration: 2
0.13367046106311992
-----
iteration: 3
0.16753257104400662
-----
iteration: 4
0.08870311757506558
-----
iteration: 5
0.14826467488117256
-----
iteration: 6
0.22869467452492875
-----
iteration: 7
0.2784235366839194
-----
iteration: 8
0.27381644730091986
-----
iteration: 9
0.24020954510889048
-----
iteration: 10
0.14662221183903187
-----
iteration: 11
0.2950992971167325
-----
iteration: 12
0.3653844643579355
-----
iteration: 13
0.2972348656020023
-----
iteration: 14
0.1153984641914775
-----
iteration: 15
0.07062397827853224
-----
iteration: 16
0.05260795743804518
-----
iteration: 17
0.07694850726055148
-----
iteration: 18
0.029193570912652823
-----
iteration: 19
0.06999962015767527
-----
iteration: 20
0.06622679619195032
-----
iteration: 21
0.04869358404871814
-----
iteration: 22
0.03470153925964393
-----
iteration: 23
0.024023744752579335
-----
iteration: 24
0.022956521719308964
-----
iteration: 25
0.02877637582622928
-----
iteration:

In [7]:
count = 0

for x in range(A.shape[0]):
    for y in range(A.shape[1]):
        if x < y:
            if A[x][y] != 0:
                count += 1

print(count)

k = 2*count/b.shape[0]

827


### MITb - Random (Cache)

In [8]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=100000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
0.7734272669917508
-----
iteration: 2
0.7734272669917508
-----
iteration: 3
0.8841555208100984
-----
iteration: 4
0.8841555208100984
-----
iteration: 5
0.5524309100764507
-----
iteration: 6
0.5524309100764507
-----
iteration: 7
0.3159730781927388
-----
iteration: 8
0.5007819470498895
-----
iteration: 9
0.39287099930354236
-----
iteration: 10
0.4009204558912507
-----
iteration: 11
0.39224286304415523
-----
iteration: 12
0.39224286304415523
-----
iteration: 13
0.39224286304415523
-----
iteration: 14
0.39140597242634706
-----
iteration: 15
0.39140597242634706
-----
iteration: 16
0.39140597242634706
-----
iteration: 17
0.39140597242634706
-----
iteration: 18
0.22508970313372154
-----
iteration: 19
0.22508970313372154
-----
iteration: 20
0.2250897031337212
-----
iteration: 21
0.2250897031337212
-----
iteration: 22
0.2250897031337212
-----
iteration: 23
0.2250897031337212
-----
iteration: 24
0.15327817836513866
-----
iteration: 25
0.22279044416356392
-----
iteration: 26
0.222790

### MITb - Fixed (Cache)

In [9]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    node_update_schedule = np.arange(A.shape[0], dtype=np.int64)
    np.random.shuffle(node_update_schedule)
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
    # print(f"streams = {iteration}")
    # print(f"node_update_schedule = {node_update_schedule}")
    sum_of_iterations += iteration

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 99 --------------
-------------- ITERATION = 2 => Streams = 97 --------------
-------------- ITERATION = 3 => Streams = 123 --------------
-------------- ITERATION = 4 => Streams = 102 --------------
-------------- ITERATION = 5 => Streams = 131 --------------
-------------- ITERATION = 6 => Streams = 118 --------------
-------------- ITERATION = 7 => Streams = 96 --------------
-------------- ITERATION = 8 => Streams = 109 --------------
-------------- ITERATION = 9 => Streams = 123 --------------
-------------- ITERATION = 10 => Streams = 108 --------------
AVE. ASYNC ITERATIONS = 110.6


### MITb - Residual (Cache)

In [10]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

-------------- ITERATION = 1 => Streams = 91 --------------
-------------- ITERATION = 2 => Streams = 117 --------------
-------------- ITERATION = 3 => Streams = 115 --------------
-------------- ITERATION = 4 => Streams = 99 --------------
-------------- ITERATION = 5 => Streams = 123 --------------
-------------- ITERATION = 6 => Streams = 118 --------------
-------------- ITERATION = 7 => Streams = 98 --------------
-------------- ITERATION = 8 => Streams = 117 --------------
-------------- ITERATION = 9 => Streams = 134 --------------
-------------- ITERATION = 10 => Streams = 126 --------------
AVE. ASYNC ITERATIONS = 113.8
